### GRU总览

GRU 于普通的 RNN 的区别在于对于潜变量的更新

GRU 在对潜变量进行更新时会考虑对历史序列信息的保留程度（这个部分也需要设计为模型参数进行学习）

引入了更新门、重置门和候选潜变量的概念，这三个可以理解为用于计算最终更新的潜变量的中间值

更新门（Update Gate）：
$$
z_t = \sigma(W_z x_t + U_z h_{t-1} + b_z)
$$

重置门（Reset Gate）：
$$
r_t = \sigma(W_r x_t + U_r h_{t-1} + b_r)
$$

候选隐藏状态（Candidate Hidden State）：
$$
\tilde{h}_t = \tanh(W_h x_t + U_h (r_t \odot h_{t-1}) + b_h)
$$

最终隐藏状态（New Hidden State）：
$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
$$


使用 GRU 更新一次潜变量的图解如下：

![](md-img/GRU.jpg)

其中箭头表示全连接

对于更新门和重置门的激活函数使用 Sigmoid 函数

对于候选潜变量的激活函数使用 tanh 函数

使用最终得到的潜变量即可进行下一个 token 的预测（同 RNN）

$$
其中 H、Z、R、\tilde{H} 形状一致
$$

<br>

### 代码从零实现

In [ ]:
import torch

class GRUModel:
    # 根据输入的词表大小和指定的隐藏层大小来初始化模型参数（要保留梯度）
    def __init__(self, vocab_size, hiden_size):
        self.vocab_size = vocab_size
        self.hiden_size = hiden_size

        self.w_hz = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xz = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_z = torch.zeros(hiden_size)
        self.w_hr = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xr = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_r = torch.zeros(hiden_size)
        self.w_hrh = torch.randn(hiden_size, hiden_size) * 0.01
        self.w_xh = torch.randn(vocab_size, hiden_size) * 0.01
        self.b_h = torch.zeros(hiden_size)
        self.w_hy = torch.randn(hiden_size, vocab_size) * 0.01
        self.b_y = torch.zeros(vocab_size)

        self.parameters = [self.w_hz, self.w_xz, self.b_z,
                           self.w_hr, self.w_xr, self.b_r,
                           self.w_hrh, self.w_xh, self.b_h,
                           self.w_hy, self.b_y]
        
        
        for param in self.parameters:
            param.requires_grad_(True)

    # 外界获取当前模型参数
    def get_parameters(self):
        return self.parameters
    
    # 此处指定输入的 X 的形状为 (time_step, batch_size, vocab_size)
    def forward(self, X):
        h = torch.zeros(x.shape[1], self.hiden_size)   # 初始化隐藏状态

        Y = []   # 用于保存所有的预测输出

        # 按找时间步长，往后推算每一个样本的潜变量
        for x in X:
            z = torch.sigmoid(x @ self.w_xz + h @ self.w_hz + self.b_z)        # 更新门
            r = torch.sigmoid(x @ self.w_xr + h @ self.w_hr + self.b_r)        # 重置门
            h_ = torch.tanh(x @ self.w_xh + (h * r) @ self.w_hrh + self.b_h)   # 候选隐藏状态

            h = z * h_ + (1 - z) * h    # 更新隐藏状态

            # 使用隐藏状态获取最终输出
            output = h @ self.w_hy + self.b_y
            Y.append(output)

        # 此处返回的 Y 的形状为 (time_step * batch_size, vocab_size)，方便计算损失函数
        return torch.cat(Y, dim=0), (h,)